In [1]:
using Agents
using Random
using InteractiveDynamics
using CairoMakie

@agent tcell ContinuousAgent{2} begin
    activation_status::Float64
    has_been_activated::Bool
    time_after_activation::Int
    is_currently_activated::Bool
end

@agent dcell ContinuousAgent{2} begin
    activation_status::Float64
    time_after_activation::Int
    is_currently_activated::Bool
end

cells = Union{tcell, dcell}

function initialize_model(; 
    n_dcell = 30, 
    n_tcell = 40, 
    speed = 2,
    extent = (900,900),    
    visual_distance = 5.0,
    activation_status = 0.0,
     has_been_activated = false,
    time_after_activation = 0, 
    is_currently_activated = false,
    )

    space2d = ContinuousSpace(extent; spacing = 10,)
    
    model = ABM(cells, space2d ,rng = MersenneTwister(30),properties = Dict(:dt => 1.0); 
        scheduler = Schedulers.randomly)

    #scale the random number
    
    for _ in 1:n_tcell
        pos = Tuple(rand(model.rng, 2)).*300
        vel = sincos(2π * rand(model.rng)) .* speed 
        add_agent!(
            pos, tcell, model , vel, activation_status, has_been_activated, time_after_activation, is_currently_activated
        )
    end
    
    for _ in 1:n_dcell
        pos = Tuple(rand(model.rng, 2)).*300
        vel = sincos(2π * rand(model.rng)) .* speed .* 0.6
        add_agent!(
            pos, dcell, model, vel, activation_status, time_after_activation, is_currently_activated
        )
    end

    return model
end

model = initialize_model()

#1 time step = 1 minute
# T cell - D cell interaction lasts for 1440 time step
function agent_step!(cells, model; speed = 5, )
    
    if cells isa dcell #d cell case
        neighbor_ids = nearby_ids(cells,model,5)  # set boundary for neighbor as 5
        
        for id in neighbor_ids
            neighbor = model[id]

            if cells.time_after_activation >= 1440
                cells.vel = sincos(2π * rand(model.rng)) .* speed .* 0.6  #random walk movement
                cells.time_after_activation = 0
            end

            if neighbor isa tcell && neighbor.has_been_activated == false
                cells.vel = (0,0)
                cells.time_after_activation += 1
                break
            else
                cells.vel = sincos(2π * rand(model.rng)) .* speed .* 0.6  #random walk movement
                cells.time_after_activation = 0
            end
        
        end   
    
    else  #t cell case
        cells.vel = sincos(2π * rand(model.rng)) .* speed #random walk movement
        neighbor_ids = nearby_ids(cells,model,5)
        
        if cells.activation_status >= 3
                cells.is_currently_activated = true
        else
            cells.is_currently_activated = false    
        end
        
#         if cells.id ==1
#             print(cells.activation_status)
#             print('\n')
#         end
        
        for id in neighbor_ids
            neighbor = model[id]
            
            if neighbor isa dcell && cells.has_been_activated == false
                   cells.has_been_activated = true
                   cells.vel = (0,0)
                   cells.activation_status += 1/240
                   cells.time_after_activation += 1
                   break

            elseif neighbor isa dcell && cells.has_been_activated == true &&cells.time_after_activation< 1440
                    cells.vel = (0,0)
                    cells.activation_status += 1/240
                    cells.time_after_activation += 1
                    break
                 
            elseif cells.time_after_activation >= 1440
                    cells.vel = sincos(2π * rand(model.rng)) .* speed
                    cells.activation_status -= 1/60
                    cells.time_after_activation += 1
                    break
            end    
        end
    end
    
    move_agent!(cells, model, model.dt)
end

CairoMakie.activate!()

istcell(a) = a isa tcell
isdcell(a) = a isa dcell
iscurrently_activated(a) = a.is_currently_activated == true

steps = 100000
adata = [(istcell,count),(isdcell,count), (iscurrently_activated, count)]
adf, mdf= run!(model, agent_step!, steps; adata)


function plot_population(adf, mdf)
    figure = Figure(resolution = (600, 400))
    ax = figure[1, 1] = Axis(figure; xlabel = "Step", ylabel = "Population")
    tcell_l = lines!(ax, adf.step, adf.count_istcell, color = :black)
    dcell_l = lines!(ax, adf.step, adf.count_isdcell, color = :blue)
    activated_l = lines!(ax, adf.step, adf.count_iscurrently_activated, color = :orange)
    figure[1, 2] = Legend(figure, [tcell_l, dcell_l, activated_l], ["tcell","dcell","activated"])
    figure
end

plot_population(adf, mdf)
    

# offset(a) = a isa tcell ? (-0.1, -0.1*rand()) : (+0.1, +0.1*rand())
ashape(a) = :circle
asize(a) = a isa dcell ? 20 : 10
acolor(a) = a.activation_status > 0 ? :blue : :black
# ac(dcell) = :blue
# ac(tcell) = :black

abmvideo("tcell.mp4", model, agent_step!;
title = "immune cell", framerate  = 100, 
    frames = 2880, as=asize, am = ashape, ac = acolor)

LoadError: InterruptException: